# Instacart Database Creation with PySqlite

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3

#pd.set_option('max_columns', 180)
#pd.set_option('max_rows', 2000000)
#pd.set_option('max_colwidth', 5000)

csvs = ['order_products__prior.csv', 'orders.csv', 'products.csv', 'aisles.csv']

df_list = []

for i in np.arange(0,len(csvs)):
    df_list.append(pd.read_csv("data/{}".format(csvs[i]),low_memory=False))
     
for d in np.arange(0,len(df_list)):
    print('#########################{}#################################'.format(csvs[d]))
    print(df_list[d].shape)
    print(df_list[d].head(2))
    print('############################################################################')

prior = df_list[0]
orders = df_list[1]
products = df_list[2]
aisles = df_list[3]

#########################order_products__prior.csv#################################
(32434489, 4)
   order_id  product_id  add_to_cart_order  reordered
0         2       33120                  1          1
1         2       28985                  2          1
############################################################################
#########################orders.csv#################################
(3421083, 7)
   order_id  user_id eval_set  order_number  order_dow  order_hour_of_day  \
0   2539329        1    prior             1          2                  8   
1   2398795        1    prior             2          3                  7   

   days_since_prior_order  
0                     NaN  
1                    15.0  
############################################################################
#########################products.csv#################################
(49688, 4)
   product_id                product_name  aisle_id  department_id
0           1  Chocolate Sandwich Cooki

In [2]:
for i in csvs:
    print i
    print df_list[csvs.index(i)].dtypes
    print '\n'

order_products__prior.csv
order_id             int64
product_id           int64
add_to_cart_order    int64
reordered            int64
dtype: object


orders.csv
order_id                    int64
user_id                     int64
eval_set                   object
order_number                int64
order_dow                   int64
order_hour_of_day           int64
days_since_prior_order    float64
dtype: object


products.csv
product_id        int64
product_name     object
aisle_id          int64
department_id     int64
dtype: object


aisles.csv
aisle_id     int64
aisle       object
dtype: object




In [25]:
DB = 'instacart.db'

def run_query(q):
    with sqlite3.connect(DB) as conn:
        return pd.read_sql(q,conn)

def run_command(c):
    with sqlite3.connect(DB) as conn:
        conn.execute('PRAGMA foreign_keys = ON;')
        conn.isolation_level = None
        conn.execute(c)

def show_tables():
    q = '''
    SELECT
        name,
        type
    FROM sqlite_master
    WHERE type IN ("table","view");
    '''
    return run_query(q)

tables = {
    "prior": prior,
    "orders": orders,
    "products": products,
    "aisles": aisles
}

with sqlite3.connect(DB) as conn:    
    for name, data in tables.items():
        conn.execute("DROP TABLE IF EXISTS {};".format(name))
        conn.text_factory = str
        data.to_sql(name,conn,index=False)
show_tables()

,name,type
0,prior,table
1,products,table
2,orders,table
3,aisles,table


In [26]:
## create aisles table
c1 = """
CREATE TABLE IF NOT EXISTS aisles1 (
    aisle_id INTEGER PRIMARY KEY,
    aisle TEXT
);
"""
## add data from csv
c2 = """
INSERT OR IGNORE INTO aisles1
SELECT
    aisle_id,
    aisle
FROM aisles;
"""
c3 = """
DROP TABLE aisles;
"""

c4="""
ALTER TABLE aisles1 RENAME TO aisles;
"""

q = """
SELECT * FROM aisles
ORDER BY 1
LIMIT 5
;
"""

for c in [c1,c2,c3,c4]:
    run_command(c)

run_query(q)

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


In [27]:
## create products table
c1 = """
CREATE TABLE IF NOT EXISTS products1 (
    product_id INTEGER PRIMARY KEY,
    product_name TEXT,
    aisle_id INTEGER,
    department_id INTEGER,
    FOREIGN KEY (aisle_id) REFERENCES aisles(aisle_id)
);
"""
## add data from csv
c2 = """
INSERT OR IGNORE INTO products1
SELECT
    product_id,
    product_name,
    aisle_id,
    department_id
FROM products;
"""
c3 = """
DROP TABLE products;
"""

c4="""
ALTER TABLE products1 RENAME TO products;
"""

q = """
SELECT * FROM products
ORDER BY 1
LIMIT 5
;
"""

for c in [c1,c2,c3,c4]:
    run_command(c)

run_query(q)

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [28]:
q =  """PRAGMA table_info(orders);"""
run_query(q)

,cid,name,type,notnull,dflt_value,pk
0,0,order_id,INTEGER,0,None,0
1,1,user_id,INTEGER,0,None,0
2,2,eval_set,TEXT,0,None,0
3,3,order_number,INTEGER,0,None,0
4,4,order_dow,INTEGER,0,None,0
5,5,order_hour_of_day,INTEGER,0,None,0
6,6,days_since_prior_order,REAL,0,None,0


In [29]:
## create products table
c1 = """
CREATE TABLE IF NOT EXISTS orders1 (
    order_id INTEGER PRIMARY KEY,
    user_id INTEGER,
    eval_set TEXT,
    order_number INTEGER,
    order_dow INTEGER,
    order_hour_of_day INTEGER,
    days_since_prior_order INTEGER
);
"""
## add data from csv
c2 = """
INSERT OR IGNORE INTO orders1
SELECT
    order_id,
    user_id,
    eval_set,
    order_number,
    order_dow,
    order_hour_of_day,
    days_since_prior_order
FROM orders;
"""
c3 = """
DROP TABLE orders;
"""

c4="""
ALTER TABLE orders1 RENAME TO orders;
"""

q = """
SELECT * FROM orders
ORDER BY 1
LIMIT 5
;
"""

for c in [c1,c2,c3,c4]:
    run_command(c)

run_query(q)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,1,112108,train,4,4,10,9
1,2,202279,prior,3,5,9,8
2,3,205970,prior,16,5,17,12
3,4,178520,prior,36,1,9,7
4,5,156122,prior,42,6,16,9


In [30]:
q =  """PRAGMA table_info(prior);"""
run_query(q)

,cid,name,type,notnull,dflt_value,pk
0,0,order_id,INTEGER,0,None,0
1,1,product_id,INTEGER,0,None,0
2,2,add_to_cart_order,INTEGER,0,None,0
3,3,reordered,INTEGER,0,None,0


In [31]:
## create prior table
c1 = """
CREATE TABLE IF NOT EXISTS prior1 (
    order_id INTEGER PRIMARY KEY,
    product_id INTEGER,
    add_to_cart_order INTEGER,
    reordered INTEGER,
    FOREIGN KEY (order_id) REFERENCES orders(order_id),
    FOREIGN KEY (product_id) REFERENCES products(product_id)
);
"""
## add data from csv
c2 = """
INSERT OR IGNORE INTO prior1
SELECT
    order_id,
    product_id,
    add_to_cart_order,
    reordered
FROM prior;
"""
c3 = """
DROP TABLE prior;
"""

c4="""
ALTER TABLE prior1 RENAME TO prior;
"""

q = """
SELECT * FROM prior
ORDER BY 1
LIMIT 5
;
"""

for c in [c1,c2,c3,c4]:
    run_command(c)

run_query(q)

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,3,33754,1,1
2,4,46842,1,0
3,5,13176,1,1
4,6,40462,1,0
